This library uses NOAA metadata on station locations to identify stations in area and request those stations from the NOAA API 

In [36]:
import requests
import pandas as pd
import io
from shapely.geometry import Point, Polygon
import re


#takes the list of stations and fire location to return stations within a degree of distance from fire
def stations_in_radius(lat,lon,radius,num_stations):
  event_location = Point(lat,lon)
  radius_center = event_location.centroid

  file = open('./emshr_lite.txt', 'r')
  count = 0
  closest_station_list = []
  while True:
    line = file.readline()
    if not line: #check for end of file 
      break
  
    count += 1
    if count < 3: #insure that only the lines with data are scraped (only past the third index)
      continue
  
    station_id = line[74:85]
    station_coordinate_lat = line[272:281].strip()
    station_coordinate_lon = line[282:292].strip()

    if station_coordinate_lat == '' or station_coordinate_lon == '' or station_id.strip() == '':
      continue

    station_coordinate_lat = float(station_coordinate_lat)
    station_coordinate_lon = float(station_coordinate_lon)
  
    station_coordinate = (station_coordinate_lat, station_coordinate_lon)
    station_coordinate = Point(station_coordinate)

    if station_coordinate.distance(radius_center) < radius:
      closest_station_list.append((station_id, station_coordinate.distance(radius_center)))
    
  closest_station_list.sort(key=lambda closest_station_list: closest_station_list[1])
  closest_station_list = closest_station_list[:num_stations]
  file.close()
  
  return [x[0] for x in closest_station_list]


#takes a string of closest stations and requests their data from noaa api
#add loop to allow for more stations to be requested 
def request_station_data(closest_stations,num_stations):
  
  df = pd.DataFrame()

  request_stations = ','.join(closest_stations)
  header = {'token': 'SXCObXMuhoovZxrVZpurQqmsHdvJWyUJ'}
  r=requests.get(f'https://www.ncei.noaa.gov/access/services/data/v1?dataset=global-summary-of-the-year&stations={request_stations}&&startDate=1950-01-01T00:00:00z&&endDate=2022-01-01&format=csv&bbox=-124.40959,32.534156,-114.131211,42.009518', headers=header)
  if r.status_code != 200:
    print(r.status_code)
    print(r.text)
    
  add_df = pd.read_csv(io.StringIO(r.text))

  #code to add the rows to the new dataframe to return 
  return add_df


#takes the station data returns a dataframe row of the averaged values for year of fire 
#make sure to take care of the missing values as well 

def filter_station_data_year(station_df, event_year):
  station_df = station_df[station_df['DATE'] == event_year]
  return station_df


#function that returns the average local climate data for year of the fire as a row to add to dataset 
def yearly_event_climate_data(lat,lon, event_datetime,radius,num_stations): 
  close_stations = stations_in_radius(lat,lon, radius, num_stations)
  stations_df = request_station_data(close_stations, num_stations)
  filtered_df = filter_station_data_year(stations_df,event_datetime)
  return filtered_df

,STATION,DATE,AWND,CDSD,CLDD,DP01,DP05,DP10,DP1X,DSND,...,WDF2,WDF5,WDFG,WDFM,WDMV,WSF1,WSF2,WSF5,WSFG,WSFM
70,ARM00087416,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,ARM00087506,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108,ARM00087509,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120,ARM00087311,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
